In [ ]:
!pip install --upgrade --quiet pinecone pinecone-text pinecone-notebooks
pip install sentence-transformers

In [6]:
api_key="pcsk_3NYGzx_Aj7YXWdwPU9v1m7w8NvxWTHh7bXXcwQpw2nFiZ86FkQULm1ugXUvSMztmDhYTQN"

In [7]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [8]:
import os
from pinecone import Pinecone,ServerlessSpec


In [ ]:
index_name="hybrid-search-langchain"
pc=Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

    

In [19]:
index = pc.Index(index_name)
index

In [14]:
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

v:\GenAI\LangChain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
from pinecone_text.sparse import BM25Encoder
bmEnc = BM25Encoder().default()
bmEnc

In [17]:
sentences= [
    "In 2025, I visited Germany",
    "In 2027, I went to Barcelona,Spain",
    "In 2024, I went to Guwahati"
]

In [18]:
bmEnc.fit(sentences)
bmEnc.dump("bm25_values.json")
bmEnc = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 15.70it/s]


In [20]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bmEnc, index=index)

In [21]:
retriever.add_texts(
    [
        "In 2025, I visited Germany",
        "In 2027, I went to Barcelona,Spain",
        "In 2024, I went to Guwahati"
    ]
)

100%|██████████| 1/1 [00:02<00:00,  2.25s/it]


In [22]:
retriever.invoke("Which year did I visit Barcelona?")

[Document(metadata={'score': 0.165076256}, page_content='In 2024, I went to Guwahati'),
 Document(metadata={'score': 0.316332728}, page_content='In 2025, I visited Germany'),
 Document(metadata={'score': 0.421359718}, page_content='In 2027, I went to Barcelona,Spain')]